In [70]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, to_date, year, count, desc, rank, sum as _sum, round

spark = SparkSession \
    .builder \
    .appName("DataFrame query 2 execution") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# Implementation 1: DataFrame API
# DataFrame Schema Definition
Crime_data_schema = StructType([
    StructField("DR_NO", IntegerType()),
    StructField("Date Rptd", StringType()),
    StructField("DATE OCC", StringType()),
    StructField("TIME OCC", IntegerType()),
    StructField("AREA", IntegerType()),
    StructField("AREA NAME", StringType()),
    StructField("Rpt Dist No", IntegerType()),
    StructField("Part 1-2", IntegerType()),
    StructField("Crm Cd", IntegerType()),
    StructField("Crm Cd Desc", StringType()),
    StructField("Mocodes", StringType()),
    StructField("Vict Age", IntegerType()),
    StructField("Vict Sex", StringType()),
    StructField("Vict Descent", StringType()),
    StructField("Premis Cd", IntegerType()),
    StructField("Premis Desc", StringType()),
    StructField("Weapon Used Cd", IntegerType()),
    StructField("Weapon Desc", StringType()),
    StructField("Status", StringType()),
    StructField("Status Desc", StringType()),
    StructField("Crm Cd 1", IntegerType()),
    StructField("Crm Cd 2", IntegerType()),
    StructField("Crm Cd 3", IntegerType()),
    StructField("Crm Cd 4", IntegerType()),
    StructField("LOCATION", StringType()),
    StructField("Cross Street", StringType()),
    StructField("LAT", FloatType()),
    StructField("LON", FloatType()),
])

RE_codes_schema = StructType([
    StructField("Vict Descent", StringType()),
    StructField("Vict Descent Full", StringType()),
])

Recent_crime_data_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", \
                                      header = True, \
                                      schema = Crime_data_schema)
Older_crime_data_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv", \
                                     header = True, \
                                     schema = Crime_data_schema)
Crime_df = Recent_crime_data_df.union(Older_crime_data_df)
RE_codes_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/RE_codes.csv", \
                             header=True, \
                             schema = RE_codes_schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
# Operations and Visualization
Crime_df_grouped = Crime_df.withColumn("year", year(to_date(col("DATE OCC"), "yyyy MMM dd hh:mm:ss a"))) \
    .fillna("X", subset=['Vict Descent']) \
    .groupBy("year", "Vict Descent").agg(count("*").alias("count"))
window_year = Window.partitionBy("year")
window_rank = Window.partitionBy("year").orderBy(desc("count"))
Crime_df_stats = Crime_df_grouped.withColumn("total_year", _sum("count").over(window_year)) \
    .withColumn("percentage", round((col("count") / col("total_year")) * 100, 1)) \
    .withColumn("rank", rank().over(window_rank))
Crime_df_final = Crime_df_stats.filter(col("rank") <= 3) \
    .join(RE_codes_df, Crime_df_stats["Vict Descent"] == RE_codes_df["Vict Descent"], "left") \
    .select(
        col("year"),
        col("Vict Descent Full").alias("Victim Descent"),
        col("count").alias("#"),
        col("percentage").alias("%")
    ) \
    .orderBy(desc("year"), desc("#")) \
    .show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------------+-----+----+
|year|Victim Descent        |#    |%   |
+----+----------------------+-----+----+
|2025|Unknown               |37   |38.1|
|2025|Hispanic/Latin/Mexican|34   |35.1|
|2025|White                 |13   |13.4|
|2024|Unknown               |49188|38.6|
|2024|Hispanic/Latin/Mexican|28576|22.4|
|2024|White                 |22958|18.0|
|2023|Hispanic/Latin/Mexican|69401|29.9|
|2023|Unknown               |59529|25.6|
|2023|White                 |44615|19.2|
|2022|Hispanic/Latin/Mexican|73111|31.1|
|2022|Unknown               |52130|22.2|
|2022|White                 |46695|19.8|
|2021|Hispanic/Latin/Mexican|63676|30.3|
|2021|Unknown               |46499|22.2|
|2021|White                 |44523|21.2|
|2020|Hispanic/Latin/Mexican|61606|30.8|
|2020|Unknown               |43958|22.0|
|2020|White                 |42638|21.3|
|2019|Hispanic/Latin/Mexican|72458|33.1|
|2019|White                 |48863|22.3|
+----+----------------------+-----+----+
only showing top